In [52]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json
from IPython.display import HTML
import pandas as pd

pd.set_option('max_rows', 20)

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36'}

base_url = 'https://krasnodar.hh.ru'
search_url = '/search/vacancy'
url_params = {'area': '53', 'search_field': ['name', 'company_name', 'description'],
              'clusters': 'true', 'ored_clusters': 'true','enable_snippets': 'true',
              'text': 'руководитель'}



In [53]:
response = requests.get(base_url + search_url, params=url_params, headers=headers)


In [54]:
dom = BeautifulSoup(response.text, 'html.parser')
vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})# не понял почему всего 20 эл-тов


In [55]:
part_of_next_page = dom.find('a', {'class': "bloko-button", 'data-qa': "pager-next"})['href']
vacancies_list = []

while part_of_next_page:
    dom = BeautifulSoup(response.text, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})

    for i in vacancies:
        vac = {}
        vac['name'] = i.find('a', {'data-qa': "vacancy-serp__vacancy-title"}).get_text()
        if i.find('span', {'data-qa': "vacancy-serp__vacancy-compensation"}):
            vac['salary'] = i.find('span', {'data-qa': "vacancy-serp__vacancy-compensation"}).get_text().replace('\u202f',' ')
        
        vac['vacancy_link'] = i.find('a')['href'] # обратиться к параметру тега href это ссылка
        
        if i.find('a', {'data-qa': "vacancy-serp__vacancy-employer"}):
            vac['sourse'] = i.find('a', {'data-qa': "vacancy-serp__vacancy-employer"}).get_text().replace('&nbsp;', ' ')
        else:
            vac['sourse'] = 'Unknown'
        vacancies_list.append(vac)
        
    next_page = base_url + part_of_next_page
    response = requests.get(next_page, headers=headers)
    print(len(vacancies_list)) # на 2460 обрывается соединение, дальше запускаю следующую строку ноутбука вручную



20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [56]:
s = pd.DataFrame(vacancies_list)
s["vacancy_link"] = '<a href="' + s.vacancy_link + '">' + 'push_to_vacancy_link' + '</a>'


HTML(s.to_html(escape=False))

,name,salary,vacancy_link,sourse
0,Заместитель генерального директора по производству,от 100 000 руб.,push_to_vacancy_link,ООО КубРотоПак
1,Руководитель обособленного структурного подразделения,от 150 000 руб.,push_to_vacancy_link,ГК ПРАКТИК
2,Руководитель,от 100 000 руб.,push_to_vacancy_link,ООО Финанс Бизнес Консалт
3,Руководитель отдела продаж,80 000 – 600 000 руб.,push_to_vacancy_link,NOVA GROUP (ИП Омельченко Светлана Викторовна)
4,Директор филиала,NaN,push_to_vacancy_link,ООО Рейл-Траст
5,Управляющий торговой сети,от 80 000 руб.,push_to_vacancy_link,Хорошие Новости
6,Операционный директор,от 250 000 руб.,push_to_vacancy_link,"Доломит, производство фасадного сайдинга"
7,Руководитель отдела по работе с клиентами,от 125 000 руб.,push_to_vacancy_link,ООО Транспортные Технологии
8,Руководитель торгового сектора,74 000 – 90 000 руб.,push_to_vacancy_link,Леруа Мерлен. Гипермаркеты
9,Руководитель проекта (подарки и подарочная продукция),от 100 000 руб.,push_to_vacancy_link,ООО КФ SHOKOLAT'E


In [57]:
s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          2480 non-null   object
 1   salary        1907 non-null   object
 2   vacancy_link  2480 non-null   object
 3   sourse        2480 non-null   object
dtypes: object(4)
memory usage: 77.6+ KB


In [59]:
s.to_csv('/home/user/Рабочий стол/ИТ профессии/GeekBrains/II четверть/Parsing/Kakorin_homework_2.csv', index=False)

In [62]:
read = pd.read_csv('/home/user/Рабочий стол/ИТ профессии/GeekBrains/II четверть/Parsing/Kakorin_homework_2.csv')
HTML(read.to_html(escape=False))

,name,salary,vacancy_link,sourse
0,Заместитель генерального директора по производству,от 100 000 руб.,push_to_vacancy_link,ООО КубРотоПак
1,Руководитель обособленного структурного подразделения,от 150 000 руб.,push_to_vacancy_link,ГК ПРАКТИК
2,Руководитель,от 100 000 руб.,push_to_vacancy_link,ООО Финанс Бизнес Консалт
3,Руководитель отдела продаж,80 000 – 600 000 руб.,push_to_vacancy_link,NOVA GROUP (ИП Омельченко Светлана Викторовна)
4,Директор филиала,NaN,push_to_vacancy_link,ООО Рейл-Траст
5,Управляющий торговой сети,от 80 000 руб.,push_to_vacancy_link,Хорошие Новости
6,Операционный директор,от 250 000 руб.,push_to_vacancy_link,"Доломит, производство фасадного сайдинга"
7,Руководитель отдела по работе с клиентами,от 125 000 руб.,push_to_vacancy_link,ООО Транспортные Технологии
8,Руководитель торгового сектора,74 000 – 90 000 руб.,push_to_vacancy_link,Леруа Мерлен. Гипермаркеты
9,Руководитель проекта (подарки и подарочная продукция),от 100 000 руб.,push_to_vacancy_link,ООО КФ SHOKOLAT'E
